In [2]:
M <- matrix(c(c(-2,-1,2,1),c(3,1,-1,-3),c(-1,0,-1,2)), nrow=4)

In [3]:
pca <- function(M, scale=TRUE, bias=FALSE){
    # returned object :
    base_result = list("base"=M)
    
    # fonction qui calcule la matrice de covariance de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    covariance_matrix <- function(M){
        # fonction qui retourne la matrice des données centrées et le centre de gravité de la matrice initiale
        centered_matrix <- function(M){
            # fonction qui calcule la matrice G, centre de gravité de la matrice passée en paramètre
            gravity_center_matrix <- function(M){
                moy_T <- colMeans(M, na.rm = FALSE, dims = 1)
                M_moy_t <- t(matrix(rep(moy_T, dim(M)[1]), ncol = dim(M)[1]))
                return(M_moy_t)
            }
            G <- gravity_center_matrix(M)
            return(list("G"=G, "M_centered"=M-G))
        }
        Mc <- centered_matrix(M)
        M_temp <- t(Mc[["M_centered"]]) %*% Mc[["M_centered"]]
        S_bias <- (1 / dim(M)[1]) * M_temp
        S <- (1 / (dim(M)[1]-1)) * M_temp
        return(append(Mc, list("S"=S, "S_bias"=S_bias)))
    }
    base_result <- append(base_result, covariance_matrix(M))
    # fonction de calcul de la matrice de correlation de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    correlation_matrix <- function(S, M_centered){
        print(S)
        # fonction qui calcule la matrice diagonale des (1/écarts-type) des colonnes
        diag_ecart_type <- function(S){
            return(list("D1_div_sd"= (1/sqrt(diag(S))) * diag(dim(S)[1])))
        }
        result <- diag_ecart_type(S)
        
        # fonction qui calcule la matrice centrée réduite
        centered_reduced_matrix <- function(M_centered, D1_S){
            return(list("M_scale"= M_centered %*% D1_S))
        }
        result <- append(result, centered_reduced_matrix(M_centered, result[["D1_div_sd"]]))
        
        # calcul de la matrice de correlation
        R_temp <- t(result[["M_scale"]]) %*% result[["M_scale"]]
        R <- (1 / dim(M)[1]) * R_temp
        return(append(result, list("R"=R)))
    }
    if(scale){
        base_result <- append(base_result, correlation_matrix(base_result[["S_bias"]], base_result[["M_centered"]]))
    }
    if(bias){
        base_result[["S"]] <- base_result[["S_bias"]]
    }
    base_result[["S_bias"]] <- NULL
    
    # building PCA main component
    pca_component <- list()
    Si <- c()
    if(scale){
        Si <- base_result[["R"]]
    }else{
        Si <- base_result[["S"]]
    }
    pca_component[["Si"]] <- Si
    
    
    
    return(base_result)
}

In [4]:
print(pca(M, bias=TRUE, scale=TRUE))

     [,1] [,2] [,3]
[1,]  2.5   -3  0.5
[2,] -3.0    5 -2.0
[3,]  0.5   -2  1.5
$base
     [,1] [,2] [,3]
[1,]   -2    3   -1
[2,]   -1    1    0
[3,]    2   -1   -1
[4,]    1   -3    2

$G
     [,1] [,2] [,3]
[1,]    0    0    0
[2,]    0    0    0
[3,]    0    0    0
[4,]    0    0    0

$M_centered
     [,1] [,2] [,3]
[1,]   -2    3   -1
[2,]   -1    1    0
[3,]    2   -1   -1
[4,]    1   -3    2

$S
     [,1] [,2] [,3]
[1,]  2.5   -3  0.5
[2,] -3.0    5 -2.0
[3,]  0.5   -2  1.5

$D1_div_sd
          [,1]      [,2]      [,3]
[1,] 0.6324555 0.0000000 0.0000000
[2,] 0.0000000 0.4472136 0.0000000
[3,] 0.0000000 0.0000000 0.8164966

$M_scale
           [,1]       [,2]       [,3]
[1,] -1.2649111  1.3416408 -0.8164966
[2,] -0.6324555  0.4472136  0.0000000
[3,]  1.2649111 -0.4472136 -0.8164966
[4,]  0.6324555 -1.3416408  1.6329932

$R
           [,1]       [,2]       [,3]
[1,]  1.0000000 -0.8485281  0.2581989
[2,] -0.8485281  1.0000000 -0.7302967
[3,]  0.2581989 -0.7302967  1.0000000



In [5]:
inertia <- function(eigenvalues){
  return(sum(eigenvalues))
}

cumulative_information_caught <- function(eigenvalues, num_axes = 2, percent = T){
  return(sum(eigenvalues[1:num_axes])/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

information_caught <- function(eigenvalues, index_axe, percent = T){
  return(eigen_values[index_axe]/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

In [74]:
norm <- function(v){
    return(sqrt(sum(v**2)))
}

eigen_function <- function(M){
    QRDecomp <- function(A){
        R <- matrix(rep(0), ncol=dim(A)[1], nrow=dim(A)[1])
        Q <- matrix(rep(0), ncol=dim(A)[1], nrow=dim(A)[1])

        nv1 <- norm(t(A[,1]))
        u1 <- t(A[,1])/nv1

        R[1,1] <- nv1
        Q[,1] <- t(u1)

        for(i in 2:dim(A)[1]){
            vit <- t(A[,i])
            for(j in 1:(i-1)){
                prod <- Q[,j]%*%A[,i]
                vit <- vit - prod%*%Q[,j]
                R[i,j] <- prod
            }

            nvi <- norm(vit)
            ui <- vit/nvi
            R[i,i] <- nvi
            Q[,i] <- ui
        }
        return(list(Q, t(R)))
    }
    
    eigen_v <- function(A, accuracy){
        QR <- QRDecomp(A)
        Q <- QR[[1]]
        A <- QR[[2]]%*%QR[[1]]
        if(accuracy > 1){
            for(i in 2:accuracy){
                QR <- QRDecomp(A)
                Q <- Q*QR[[1]]
                A <- QR[[2]]%*%QR[[1]]
            }
        }
        return(diag(A))
    }
    dimM = dim(M)[1]
    eigen_values <- eigen_v(M, 10)
    eigen_vectors <- matrix(0, ncol=dimM, nrow=dimM)

    

    for(i in 1:dimM){
        print(t(M-eigen_values[i]*diag(dimM))%*%c(0,0))
    }
    return(list("eigen_values"=eigen_values, "eigen_vectors"=eigen_vectors))
}

In [75]:
A <- matrix(rep(1), ncol= 2, nrow=2)
A[1,1] <- 1
A[1,2] <- 2
A[2,1] <- 3
A[2,2] <- 4
A
t(A)
eigen_function(A)

1,2
3,4


1,3
2,4


     [,1]
[1,]    0
[2,]    0
     [,1]
[1,]    0
[2,]    0


0,0
0,0


In [19]:
eigen(A)

eigen() decomposition
$values
[1]  5.3722813 -0.3722813

$vectors
           [,1]       [,2]
[1,] -0.4159736 -0.8245648
[2,] -0.9093767  0.5657675
